In [1]:
# What : Ici j'importe les librairies nécessaires pour l'EDA démographique
# Why  : Parce que je veux tracer avec Plotly et piloter via ipywidgets
# How  : Pour ce faire, j'importe pandas, numpy, plotly.express, ipywidgets et display

import pandas as pd            # charge les données
import numpy as np             # gère les types numériques
import plotly.express as px    # trace les graphiques interactifs
import ipywidgets as widgets   # crée les widgets
from IPython.display import display, Markdown  # affiche proprement


In [2]:
# What : Ici je charge le CSV et je conserve les variables démographiques + la cible
# Why  : Parce que je veux me concentrer sur age, job, marital, education et y
# How  : Pour ce faire, je lis le CSV (sep=';'), je sous-sélectionne les colonnes, puis je prévisualise

# 1. Je définis le chemin et le séparateur
DATA_PATH = "../data/bank-full.csv"  # ajuste si besoin
SEP = ";"                            # sépare les colonnes

# 2. Je charge le dataset
df = pd.read_csv(DATA_PATH, sep=SEP)  # lit le fichier

# 3. Je conserve les colonnes d'intérêt
demo_cols = ["age", "job", "marital", "education", "y"]  # liste les colonnes
df_demo = df[demo_cols].copy()                            # crée le sous-ensemble

# 4. Je prévisualise
display(df_demo.head())  # affiche l’aperçu


,age,job,marital,education,y
0,58,management,married,tertiary,no
1,44,technician,single,secondary,no
2,33,entrepreneur,married,secondary,no
3,47,blue-collar,married,unknown,no
4,33,unknown,single,unknown,no


In [3]:
# What : Ici je visualise la distribution de chaque variable démographique
# Why  : Parce que je veux détecter forme, déséquilibres et outliers
# How  : Pour ce faire, je trace un histogramme pour age et des barres pour les catégorielles

# 1. Je trace l'histogramme de l'âge
fig_age = px.histogram(df_demo, x="age", nbins=30, title="Age — distribution")  # crée l'histo
fig_age.show()  # affiche le graphe

# 2. Je trace les barres pour job, marital, education (fix: colonnes nommées explicitement)
for col in ["job", "marital", "education"]:
    vc = (
        df_demo[col].astype(str)
        .value_counts(dropna=False)
        .reset_index(name="count")      # nomme la colonne des comptes
        .rename(columns={"index": col}) # renomme l’index en nom de la feature
    )
    fig = px.bar(
        vc, x=col, y="count",
        labels={col: col, "count": "count"},
        title=f"{col.capitalize()} — distribution"       # crée le bar chart
    )
    fig.update_layout(xaxis_tickangle=-45)  # incline les labels
    fig.show()  # affiche le graphe


In [4]:
# What : Ici je compare chaque variable démographique à la cible y
# Why  : Parce que je veux voir les patterns associés à la souscription (yes/no)
# How  : Pour ce faire, je trace un violin pour age et des barres empilées pour les catégorielles

# 1. Je trace âge vs y (violin + box)
fig_age_y = px.violin(
    df_demo, x="y", y="age", box=True, points="all",
    title="Age by Subscription (y)"   # crée le violin
)
fig_age_y.show()  # affiche le graphe

# 2. Je trace les barres empilées pour job/marital/education vs y
for col in ["job", "marital", "education"]:
    fig = px.histogram(
        df_demo, x=col, color="y", barmode="stack",
        title=f"{col.capitalize()} by Subscription (y)"  # crée l'histogramme empilé
    )
    fig.update_layout(xaxis_tickangle=-45)  # incline les labels
    fig.show()  # affiche le graphe


In [6]:
# What : Ici je crée un explorateur interactif des variables démographiques
# Why  : Parce que je veux changer de variable rapidement sans rerun plusieurs cellules
# How  : Pour ce faire, j'utilise un Dropdown ipywidgets et je mets à jour le graphe dynamiquement

# 1. Je prépare le sélecteur
var_dropdown = widgets.Dropdown(
    options=["age", "job", "marital", "education"],
    value="age",
    description="Variable"
)  # crée le dropdown

# 2. Je définis la fonction de rendu
def update_plot(var):
    is_num = pd.api.types.is_numeric_dtype(df_demo[var])  # teste le type
    if is_num:
        fig = px.histogram(df_demo, x=var, nbins=30, title=f"{var.capitalize()} — distribution")  # histo
    else:
        fig = px.histogram(df_demo, x=var, color="y", barmode="stack",
                           title=f"{var.capitalize()} by Subscription (y)")  # barres empilées
        fig.update_layout(xaxis_tickangle=-45)  # incline les labels
    fig.show()  # affiche le graphe

# 3. J'active l'interaction
widgets.interact(update_plot, var=var_dropdown)  # lance le widget


interactive(children=(Dropdown(description='Variable', options=('age', 'job', 'marital', 'education'), value='…

<function __main__.update_plot(var)>